In [2]:
import pandas as pd
import geopandas as gpd
import folium

In [3]:
df = pd.read_csv('Ulst6Jan87.csv')

In [6]:
Historic_Gravity = gpd.GeoDataFrame(df[['id', 'OBS (mgal)']], # Only display the ID for each point along with the gravity value in Milligal
                            geometry=gpd.points_from_xy(df['LONG'], df['LAT']), # set the geometry using points_from_xy
                            crs='epsg:4326') # set CRS as WGS84 lat/lon

In [7]:
Historic_Gravity.head()

,id,OBS (mgal),geometry
0,U1 1A02,981504.62,POINT (-6.20680 55.29600)
1,U1 1A03,981507.39,POINT (-6.23730 55.29550)
2,U1 1A04,981510.10,POINT (-6.25380 55.29970)
3,U1 1A05,981524.64,POINT (-6.26700 55.29270)
4,U1 1B02,981525.89,POINT (-6.18650 55.26220)


In [97]:
df = pd.read_csv('OSNIAbsoluteGravity.csv')

In [126]:
df.head()

,Unnamed: 0,Station Name,GNSS Observation Date,Easting,Northing,Elevation,Easting.1,Northing.1,Unnamed: 8,latitude,longitude,Altitude
0,1,Cushendall,27/02/2020,724301.9337,927608.7551,1.100000,324377,427624,NaN,55-4-49.235970N,6-3-11.641755W,57.076594
1,2,Bangor,09/03/2020,750484.3709,882233.5231,4.671654,350564,382238,NaN,54-39-56.749041N,5-40-1.846710W,60.425980
2,3,Belfast,24/08/2022,733995.2420,876707.6150,15.452000,334072,376711,NaN,54-37-14.868064N,5-55-30.456762W,71.575989
3,3,Belfast IGSN71,24/08/2022,736473.4086,876421.7018,1.760000,336551,376426,NaN,54-37-3.238087N,5-53-12.894233W,57.839865
4,4,Moneymore,11/03/2020,685772.5538,883608.0075,53.296145,285839,383613,NaN,54-41-35.775369N,6-40-10.307837W,110.195640


In [ ]:
AbsoluteGravity = gpd.GeoDataFrame(df[['id', 'OBS (mgal)']], # Only display the ID for each point along with the gravity value in Milligal
                            geometry=gpd.points_from_xy(df['LONG'], df['LAT']), # set the geometry using points_from_xy
                            crs='epsg:29903') # set CRS as WGS84 lat/lon


#AbsoluteGravity= gpd.GeoDataFrame(df[['Station Name']], geometry=gpd.points_from_xy(df['Easting'], df['Northing']))
#gdf = GeoDataFrame(df, geometry=geometry, crs= 29903) 
gdf = gdf.to_crs(4326) # change CRS to WGS84 lat/lon
                            

